In [ ]:
import pandas as pd
import folium
import json

In [ ]:
# Lecture des données
df_acpm = pd.read_excel('data/ACPM_list_diffusion_20251110.xlsx', sheet_name='PQR')
df_zones = pd.read_csv('data/zones_diffusion.csv', encoding='latin1', sep=';')
df_dept = pd.read_csv('data/departements_avec_regions.csv', encoding='utf-8-sig', sep=';', skiprows=1)
df_dept.columns = ['code', 'nom', 'region']

In [ ]:

print("=== Chargement des données ===")
print(f"Journaux ACPM: {len(df_acpm)}")
print(f"Journaux avec zones: {len(df_zones)}")
print(f"Départements référencés: {len(df_dept)}")
print(f"Régions référencés: {df_dept.region.nunique()}")

In [ ]:
# Normalisation des noms de départements
normalisation_dept = {
    "Côtes d'Armor": "Côtes d'Armor",
    "Nord de Meurthe-et-Moselle": "Meurthe-et-Moselle",
    "Seine-St-Denis": "Seine-Saint-Denis",
}


In [ ]:
# Créer le mapping nom -> code (en normalisant)
nom_vers_code = {}
for idx, row in df_dept.iterrows():
    code = str(row['code']).zfill(2) if row['code'] not in ['2A', '2B'] and int(row['code']) < 100 else str(row['code'])
    nom = row['nom'].strip()
    nom_vers_code[nom] = code
    # Ajouter les variantes normalisées
    if nom in normalisation_dept:
        nom_vers_code[normalisation_dept[nom]] = code

print(f"\n=== Mapping créé: {len(nom_vers_code)} entrées ===")


In [ ]:
# Fonction pour extraire les départements
def extraire_departements(zone_str):
    if pd.isna(zone_str):
        return []
    depts = [d.strip() for d in zone_str.split(',')]
    return [normalisation_dept.get(d, d) for d in depts]

df_zones['departements'] = df_zones['Zone de diffusion'].apply(extraire_departements)


In [ ]:
# Merger les données ACPM avec les zones
df_complet = df_zones.merge(
    df_acpm[['Titre', 'Diffusion France Payée', 'Evolution en %']], 
    on='Titre', 
    how='left'
)

In [ ]:
# Trier par diffusion (décroissant)
df_complet = df_complet.sort_values('Diffusion France Payée', ascending=False, na_position='last')
df_complet = df_complet.reset_index(drop=True)
df_complet['rang'] = range(1, len(df_complet) + 1)

print(f"\n{'='*80}")
print(f"{'CLASSEMENT DES JOURNAUX PQR PAR DIFFUSION':^80}")
print(f"{'(du plus diffusé au moins diffusé)':^80}")
print(f"{'='*80}\n")

for idx, row in df_complet.head(20).iterrows():
    if pd.notna(row['Diffusion France Payée']):
        medaille = ""
        if row['rang'] == 1:
            medaille = "🥇 "
        elif row['rang'] == 2:
            medaille = "🥈 "
        elif row['rang'] == 3:
            medaille = "🥉 "
        
        evolution_icon = "↗" if row['Evolution en %'] > 0 else "↘"
        print(f"{medaille}{row['rang']:2d}. {row['Titre']:<50s} {int(row['Diffusion France Payée']):>10,} ex. {evolution_icon} {row['Evolution en %']:+.1f}%")


In [ ]:
# Créer une structure de données pour chaque département
dept_data = {}

for idx, row in df_dept.iterrows():
    code = str(row['code']).zfill(2) if row['code'] not in ['2A', '2B'] and int(row['code']) < 100 else str(row['code'])
    dept_data[code] = {
        'nom': row['nom'],
        'nb_journaux': 0,
        'journaux': [],
        'diffusion_totale': 0
    }

In [ ]:



# Remplir les données par département
for idx, row in df_complet.iterrows():
    if pd.notna(row['Diffusion France Payée']):
        titre = row['Titre']
        departements = row['departements']
        diffusion = row['Diffusion France Payée']
        evolution = row['Evolution en %']
        rang = row['rang']
        
        for dept_nom in departements:
            code = nom_vers_code.get(dept_nom)
            if code and code in dept_data:
                dept_data[code]['nb_journaux'] += 1
                dept_data[code]['journaux'].append({
                    'titre': titre,
                    'diffusion': diffusion,
                    'evolution': evolution,
                    'rang': rang
                })
                dept_data[code]['diffusion_totale'] += diffusion

print(f"\n{'='*80}")
print(f"=== STATISTIQUES DÉPARTEMENTALES ===")
print(f"Départements couverts: {sum(1 for d in dept_data.values() if d['nb_journaux'] > 0)} / {len(dept_data)}")
print(f"Max journaux par département: {max(d['nb_journaux'] for d in dept_data.values())}")

# Top 10 des départements les mieux couverts
top_depts = sorted([(code, data) for code, data in dept_data.items()], 
                   key=lambda x: x[1]['nb_journaux'], reverse=True)[:10]
print(f"\nTop 10 des départements les plus couverts:")
for i, (code, data) in enumerate(top_depts, 1):
    if data['nb_journaux'] > 0:
        print(f"  {i:2d}. {data['nom']:<30s} {data['nb_journaux']} journal{'aux' if data['nb_journaux'] > 1 else ''}")

# Créer un GeoJSON simplifié pour la France
# On va utiliser l'API geo.api.gouv.fr qui est accessible
print(f"\n{'='*80}")
print("=== CRÉATION DE LA CARTE CHOROPLÈTHE ===")

# Créer la carte Folium
m = folium.Map(
    location=[46.8, 2.5],
    zoom_start=6,
    tiles='CartoDB positron',
    prefer_canvas=True
)

# URL du GeoJSON des départements depuis geo.api.gouv.fr
# Cette API publique devrait être accessible
geojson_url = 'https://geo.api.gouv.fr/departements?format=geojson&geometry=contour'

# Créer un DataFrame pour la choropleth
df_choropleth = pd.DataFrame([
    {
        'code': code,
        'nom': data['nom'],
        'nb_journaux': data['nb_journaux'],
        'diffusion_totale': data['diffusion_totale']
    }
    for code, data in dept_data.items()
])

print(f"DataFrame choropleth créé: {len(df_choropleth)} départements")

try:
    # Télécharger le GeoJSON avec requests
    import urllib.request
    
    with urllib.request.urlopen(geojson_url) as response:
        geojson_data = json.loads(response.read().decode())
    
    print(f"GeoJSON téléchargé: {len(geojson_data['features'])} départements")
    
    # Enrichir le GeoJSON avec nos données
    for feature in geojson_data['features']:
        code_dept = feature['properties']['code']
        if code_dept in dept_data:
            feature['properties'].update({
                'nb_journaux': dept_data[code_dept]['nb_journaux'],
                'journaux_info': dept_data[code_dept]['journaux'],
                'nom_dept': dept_data[code_dept]['nom'],
                'diffusion_totale': dept_data[code_dept]['diffusion_totale']
            })
        else:
            feature['properties'].update({
                'nb_journaux': 0,
                'journaux_info': [],
                'nom_dept': feature['properties'].get('nom', 'Inconnu'),
                'diffusion_totale': 0
            })
    
    # Ajouter la choropleth avec Folium
    folium.Choropleth(
        geo_data=geojson_data,
        name='Nombre de journaux PQR',
        data=df_choropleth,
        columns=['code', 'nb_journaux'],
        key_on='feature.properties.code',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.8,
        legend_name='Nombre de journaux PQR distribués par département',
        nan_fill_color='#f0f0f0',
        nan_fill_opacity=0.4,
        highlight=True
    ).add_to(m)
    
    # Ajouter une couche GeoJSON avec tooltips et popups personnalisés
    style_function = lambda x: {
        'fillOpacity': 0,
        'color': '#333333',
        'weight': 1.5
    }
    
    highlight_function = lambda x: {
        'fillColor': '#ffff00',
        'color': '#000000',
        'weight': 3,
        'fillOpacity': 0.7
    }
    
    # Créer les popups personnalisés
    def create_popup_html(feature):
        props = feature['properties']
        code = props.get('code', '')
        nom = props.get('nom_dept', props.get('nom', 'N/A'))
        nb_journaux = props.get('nb_journaux', 0)
        journaux_info = props.get('journaux_info', [])
        
        if nb_journaux == 0:
            popup_html = f"""
            <div style="min-width: 250px; font-family: Arial, sans-serif;">
                <h4 style="margin: 5px 0; color: #999; border-bottom: 2px solid #999; padding-bottom: 5px;">
                    {nom} ({code})
                </h4>
                <p style="margin: 10px 0; font-size: 13px; color: #666;">
                    ❌ Aucun journal PQR distribué dans ce département
                </p>
            </div>
            """
        else:
            popup_html = f"""
            <div style="min-width: 350px; max-width: 450px; font-family: Arial, sans-serif;">
                <h4 style="margin: 5px 0; color: #2171b5; border-bottom: 3px solid #2171b5; padding-bottom: 5px;">
                    📍 {nom} ({code})
                </h4>
                <p style="margin: 10px 0; font-size: 14px; background-color: #e8f4f8; padding: 8px; border-radius: 5px;">
                    <strong>🗞️ {nb_journaux} journal{'aux' if nb_journaux > 1 else ''} PQR distribué{'s' if nb_journaux > 1 else ''}</strong>
                </p>
                <div style="max-height: 350px; overflow-y: auto; margin-top: 10px;">
            """
            
            # Trier les journaux par rang (classement)
            journaux_sorted = sorted(journaux_info, key=lambda x: x['rang'])
            
            for j in journaux_sorted:
                evolution_icon = "📈" if j['evolution'] > 0 else "📉"
                evolution_color = "#28a745" if j['evolution'] > 0 else "#dc3545"
                
                # Couleur de badge selon le rang
                if j['rang'] == 1:
                    badge_color = "#FFD700"
                    badge_text = "🥇"
                elif j['rang'] == 2:
                    badge_color = "#C0C0C0"
                    badge_text = "🥈"
                elif j['rang'] == 3:
                    badge_color = "#CD7F32"
                    badge_text = "🥉"
                elif j['rang'] <= 10:
                    badge_color = "#4CAF50"
                    badge_text = f"#{j['rang']}"
                else:
                    badge_color = "#6c757d"
                    badge_text = f"#{j['rang']}"
                
                popup_html += f"""
                <div style="margin: 10px 0; padding: 10px; background-color: #f8f9fa; 
                            border-left: 5px solid {badge_color}; border-radius: 5px; 
                            box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <div style="display: flex; justify-content: space-between; align-items: center;">
                        <div style="font-weight: bold; color: #333; font-size: 13px; flex: 1;">
                            {j['titre']}
                        </div>
                        <div style="background-color: {badge_color}; color: #fff; 
                                    padding: 3px 8px; border-radius: 12px; font-size: 11px; 
                                    font-weight: bold; margin-left: 10px;">
                            {badge_text}
                        </div>
                    </div>
                    <div style="font-size: 11px; color: #666; margin-top: 5px; line-height: 1.6;">
                        📰 <strong>Diffusion:</strong> {int(j['diffusion']):,} exemplaires<br>
                        {evolution_icon} <strong>Évolution:</strong> 
                        <span style="color: {evolution_color}; font-weight: bold;">
                            {j['evolution']:+.2f}%
                        </span>
                    </div>
                </div>
                """
            
            popup_html += "</div></div>"
        
        return popup_html
    
    # Ajouter la couche avec popups
    geojson_layer = folium.GeoJson(
        geojson_data,
        style_function=style_function,
        highlight_function=highlight_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['nom_dept', 'nb_journaux'],
            aliases=['Département:', 'Nombre de journaux:'],
            localize=True,
            sticky=False,
            labels=True,
            style="""
                background-color: white;
                border: 2px solid black;
                border-radius: 5px;
                padding: 10px;
                font-size: 14px;
                font-weight: bold;
            """,
            max_width=300,
        )
    )
    
    # Ajouter les popups personnalisés
    for feature in geojson_layer.data['features']:
        popup_html = create_popup_html(feature)
        
        # Calculer le centroïde approximatif pour placer le popup
        # (pas nécessaire avec GeoJson, les popups s'affichent au clic)
    
    geojson_layer.add_to(m)
    
    # Titre
    title_html = '''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 750px;
                background-color: white; border: 3px solid #2171b5; z-index:9999; 
                font-size: 14px; padding: 20px; border-radius: 10px; 
                box-shadow: 0 4px 10px rgba(0,0,0,0.3);">
        <h2 style="margin: 0 0 12px 0; color: #2171b5; text-align: center;">
            🏆 Classement et Diffusion de la Presse Quotidienne Régionale
        </h2>
        <div style="background-color: #e8f4f8; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
            <p style="margin: 5px 0; font-size: 13px; text-align: center;">
                <strong>Carte Choroplèthe Interactive</strong><br>
                Couleur = Nombre de journaux PQR distribués par département
            </p>
        </div>
        <p style="margin: 5px 0; font-size: 12px;">
            💡 <strong>Navigation:</strong><br>
            • <em>Survolez</em> un département pour voir le nombre de journaux<br>
            • <em>Cliquez</em> sur un département pour voir le détail complet avec classement national
        </p>
        <p style="margin: 8px 0 0 0; font-size: 11px; color: #666; text-align: center; 
                   border-top: 1px solid #ddd; padding-top: 8px;">
            📊 Données ACPM 2025 • {nb_journaux} journaux analysés • {nb_dept} départements couverts
        </p>
    </div>
    '''.format(nb_journaux=len(df_complet), nb_dept=sum(1 for d in dept_data.values() if d['nb_journaux'] > 0))
    
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Sauvegarder la carte
    output_path = '/mnt/user-data/outputs/carte_classement_pqr_choropleth.html'
    m.save(output_path)
    print(f"✅ Carte choroplèthe sauvegardée: {output_path}")
    
except Exception as e:
    print(f"❌ Erreur lors de la création de la carte: {e}")
    import traceback
    traceback.print_exc()

# Créer le tableau HTML de classement détaillé
print("\n=== CRÉATION DU TABLEAU DE CLASSEMENT ===")

classement_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Classement PQR par Diffusion</title>
    <style>
        body {{ 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
            margin: 0; 
            padding: 20px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        }}
        .container {{
            max-width: 1800px;
            margin: 0 auto;
            background-color: white;
            padding: 40px;
            border-radius: 15px;
            box-shadow: 0 8px 30px rgba(0,0,0,0.3);
        }}
        h1 {{ 
            color: #2171b5; 
            text-align: center;
            border-bottom: 5px solid #2171b5;
            padding-bottom: 20px;
            margin-bottom: 30px;
            font-size: 32px;
        }}
        .subtitle {{
            text-align: center;
            color: #666;
            font-size: 16px;
            margin-bottom: 30px;
            font-style: italic;
        }}
        .stats-box {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
            gap: 25px;
            margin-bottom: 40px;
        }}
        .stat-card {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 25px;
            border-radius: 15px;
            text-align: center;
            box-shadow: 0 6px 12px rgba(0,0,0,0.15);
            transition: transform 0.3s;
        }}
        .stat-card:hover {{
            transform: translateY(-5px);
        }}
        .stat-number {{
            font-size: 42px;
            font-weight: bold;
            margin: 15px 0;
        }}
        .stat-label {{
            font-size: 15px;
            opacity: 0.95;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        table {{ 
            border-collapse: collapse; 
            width: 100%; 
            margin-top: 20px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            border-radius: 10px;
            overflow: hidden;
        }}
        th, td {{ 
            border: 1px solid #e0e0e0; 
            padding: 16px; 
            text-align: left; 
        }}
        th {{ 
            background-color: #2171b5; 
            color: white; 
            font-weight: bold;
            position: sticky;
            top: 0;
            z-index: 10;
            font-size: 14px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}
        tr:nth-child(even) {{ 
            background-color: #f8f9fa; 
        }}
        tr:hover {{
            background-color: #e3f2fd;
            transition: background-color 0.2s;
        }}
        .rang {{
            font-size: 22px;
            font-weight: bold;
            color: #2171b5;
            text-align: center;
            min-width: 60px;
        }}
        .medaille {{
            font-size: 28px;
        }}
        .top3 {{
            background: linear-gradient(135deg, #ffd700 0%, #ffed4e 100%) !important;
            font-weight: bold;
        }}
        .diffusion {{
            font-size: 17px;
            font-weight: bold;
            color: #333;
            text-align: right;
        }}
        .evolution {{
            text-align: center;
            font-weight: bold;
            font-size: 14px;
        }}
        .positive {{ color: #28a745; }}
        .negative {{ color: #dc3545; }}
        .zone-cell {{
            font-size: 12px;
            max-width: 450px;
            line-height: 1.5;
            color: #555;
        }}
        .groupe-badge {{
            display: inline-block;
            padding: 5px 12px;
            border-radius: 15px;
            font-size: 11px;
            background-color: #6c757d;
            color: white;
            font-weight: bold;
        }}
        .info-box {{
            margin-top: 40px;
            padding: 25px;
            background-color: #f8f9fa;
            border-radius: 10px;
            border-left: 6px solid #2171b5;
        }}
        .info-box h3 {{
            margin-top: 0;
            color: #2171b5;
            font-size: 22px;
        }}
        .info-box ul {{
            line-height: 2;
            font-size: 15px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>🏆 CLASSEMENT DES JOURNAUX PQR PAR DIFFUSION</h1>
        <p class="subtitle">Du plus diffusé au moins diffusé • Données ACPM 2025</p>
        
        <div class="stats-box">
            <div class="stat-card">
                <div class="stat-label">📰 Diffusion Totale</div>
                <div class="stat-number">{df_complet['Diffusion France Payée'].sum():,.0f}</div>
                <div class="stat-label">exemplaires</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">🗞️ Journaux Analysés</div>
                <div class="stat-number">{len(df_complet)}</div>
                <div class="stat-label">titres PQR</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">🗺️ Départements Couverts</div>
                <div class="stat-number">{sum(1 for d in dept_data.values() if d['nb_journaux'] > 0)}</div>
                <div class="stat-label">sur 101</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">📊 Couverture Moyenne</div>
                <div class="stat-number">{sum(d['nb_journaux'] for d in dept_data.values()) / sum(1 for d in dept_data.values() if d['nb_journaux'] > 0):.1f}</div>
                <div class="stat-label">journaux / département</div>
            </div>
        </div>
        
        <table>
            <thead>
                <tr>
                    <th style="width: 90px;">Rang</th>
                    <th>Titre</th>
                    <th style="width: 180px;">Groupe</th>
                    <th style="width: 170px;">Diffusion</th>
                    <th style="width: 130px;">Évolution</th>
                    <th style="width: 110px;">Nb Dép.</th>
                    <th>Zone de diffusion</th>
                </tr>
            </thead>
            <tbody>
"""

for idx, row in df_complet.iterrows():
    if pd.notna(row['Diffusion France Payée']):
        rang = row['rang']
        
        # Médailles pour le top 3
        medaille = ""
        row_class = ""
        if rang == 1:
            medaille = '<span class="medaille">🥇</span> '
            row_class = 'top3'
        elif rang == 2:
            medaille = '<span class="medaille">🥈</span> '
            row_class = 'top3'
        elif rang == 3:
            medaille = '<span class="medaille">🥉</span> '
            row_class = 'top3'
        
        evolution_class = "positive" if row['Evolution en %'] > 0 else "negative"
        evolution_symbol = "↗" if row['Evolution en %'] > 0 else "↘"
        
        zone = row['Zone de diffusion'] if pd.notna(row['Zone de diffusion']) else 'N/A'
        if len(str(zone)) > 180:
            zone = str(zone)[:180] + "..."
        
        groupe = row['Groupe'] if pd.notna(row['Groupe']) else 'N/A'
        nb_dept = len(row['departements'])
        
        classement_html += f"""
            <tr class="{row_class}">
                <td class="rang">{medaille}{rang}</td>
                <td><strong style="font-size: 14px;">{row['Titre']}</strong></td>
                <td><span class="groupe-badge">{groupe}</span></td>
                <td class="diffusion">{int(row['Diffusion France Payée']):,}</td>
                <td class="evolution {evolution_class}">{evolution_symbol} {row['Evolution en %']:+.2f}%</td>
                <td style="text-align: center; font-weight: bold; font-size: 16px; color: #2171b5;">{nb_dept}</td>
                <td class="zone-cell">{zone}</td>
            </tr>
        """

classement_html += f"""
            </tbody>
        </table>
        
        <div class="info-box">
            <h3>📌 Légende et Notes</h3>
            <ul>
                <li><strong>Classement par diffusion décroissante</strong> : Les journaux sont ordonnés du plus diffusé (#1) au moins diffusé</li>
                <li><strong>Diffusion</strong> : Nombre d'exemplaires payés distribués en France (données ACPM)</li>
                <li><strong>Évolution</strong> : Variation en % par rapport à la période précédente (↗ hausse, ↘ baisse)</li>
                <li><strong>Nb Dép.</strong> : Nombre de départements dans lesquels le journal est distribué</li>
                <li><strong>Podium</strong> : 🥇 1er • 🥈 2ème • 🥉 3ème (fond doré)</li>
            </ul>
        </div>
        
        <div style="margin-top: 40px; text-align: center; color: #666; font-size: 13px; 
                    padding: 20px; border-top: 2px solid #e0e0e0;">
            📊 <strong>Données ACPM</strong> • Analyse réalisée le 10 novembre 2025<br>
            💻 Visualisation interactive disponible dans le fichier carte_classement_pqr_choropleth.html
        </div>
    </div>
</body>
</html>
"""

with open('/mnt/user-data/outputs/classement_detaille_pqr.html', 'w', encoding='utf-8') as f:
    f.write(classement_html)

print(f"✅ Tableau de classement détaillé sauvegardé")

# Export CSV du classement
export_classement = df_complet[df_complet['Diffusion France Payée'].notna()].copy()
export_classement['Nb_departements'] = export_classement['departements'].apply(len)
export_classement_final = export_classement[['rang', 'Titre', 'Groupe', 'Diffusion France Payée', 
                                              'Evolution en %', 'Nb_departements', 'Zone de diffusion']].copy()
export_classement_final.columns = ['Rang', 'Titre', 'Groupe', 'Diffusion', 'Evolution_pct', 
                                    'Nb_départements', 'Zone_de_diffusion']
export_classement_final.to_csv('/mnt/user-data/outputs/classement_pqr_detaille.csv', 
                                index=False, encoding='utf-8-sig')

print(f"✅ Export CSV du classement sauvegardé")

# Export CSV des données par département
dept_export = []
for code, data in dept_data.items():
    if data['nb_journaux'] > 0:
        journaux_list = [j['titre'] for j in sorted(data['journaux'], key=lambda x: x['rang'])]
        dept_export.append({
            'Code': code,
            'Département': data['nom'],
            'Nb_journaux': data['nb_journaux'],
            'Journaux': ', '.join(journaux_list),
            'Diffusion_totale': int(data['diffusion_totale'])
        })

df_dept_export = pd.DataFrame(dept_export).sort_values('Nb_journaux', ascending=False)
df_dept_export.to_csv('/mnt/user-data/outputs/departements_pqr.csv', index=False, encoding='utf-8-sig')

print(f"✅ Export CSV des données départementales sauvegardé")

print(f"\n{'='*80}")
print("✨ TRAITEMENT TERMINÉ AVEC SUCCÈS ✨")
print(f"{'='*80}")
print("\n📁 Fichiers générés:")
print("  1. carte_classement_pqr_choropleth.html - Carte interactive choroplèthe")
print("  2. classement_detaille_pqr.html - Tableau de classement complet")
print("  3. classement_pqr_detaille.csv - Export CSV du classement")
print("  4. departements_pqr.csv - Export CSV des données par département")